In [1]:
!python -V

Python 3.9.12


In [16]:
import os
import sys
# Get the parent directory and add it to the system path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)

In [20]:
import pickle

from env import *
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [35]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [25]:
import mlflow
import mlflow.experiments


mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("nyc-taxi-experiment")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://127.0.0.1:5000'


In [27]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)
    
    return df

In [28]:
df_train = read_dataframe("../data/hw2/train/green_tripdata_2023-01.parquet")
df_val = read_dataframe("../data/hw2/train/green_tripdata_2023-02.parquet")

In [29]:
len(df_train), len(df_val)

(65946, 62574)

In [30]:
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [31]:
categorical = ["PU_DO"] #"PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [32]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [36]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

6.03727552054262

In [38]:
with open("../models/hw2/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [40]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Joses")

    mlflow.log_param("train-data-path", "../data/hw2/train/green_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "../data/hw2/train/green_tripdata_2023-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="../models/hw2/lin_reg.bin", artifact_path="models_pickle")

In [42]:
import xgboost as xgb

In [43]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [44]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [47]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status": STATUS_OK}

In [48]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/josesmac/MLOps_Projects/mlops_zoomcamp_24/zmcp/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:01:14] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.71629                           
[1]	validation-rmse:8.18760                           
[2]	validation-rmse:7.73905                           
[3]	validation-rmse:7.34636                           
[4]	validation-rmse:7.02017                           
[5]	validation-rmse:6.74074                           
[6]	validation-rmse:6.49738                           
[7]	validation-rmse:6.29834                           
[8]	validation-rmse:6.13069                           
[9]	validation-rmse:5.98214                           
[10]	validation-rmse:5.87214                          
[11]	validation-rmse:5.76980                          
[12]	validation-rmse:5.69545                          
[13]	validation-rmse:5.62729                          
[14]	validation-rmse:5.57344                          
[15]	validation-rmse:5.52257                          
[16]	validation-rmse:5.48541                          
[17]	validation-rmse:5.45495                          
[18]	valid

/Users/josesmac/MLOps_Projects/mlops_zoomcamp_24/zmcp/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:01:56] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.82028                                                    
[1]	validation-rmse:8.37392                                                    
[2]	validation-rmse:7.97776                                                    
[3]	validation-rmse:7.62776                                                    
[4]	validation-rmse:7.31912                                                    
[5]	validation-rmse:7.04860                                                    
[6]	validation-rmse:6.81212                                                    
[7]	validation-rmse:6.60563                                                    
[8]	validation-rmse:6.42547                                                    
[9]	validation-rmse:6.26973                                                    
[10]	validation-rmse:6.13487                                                   
[11]	validation-rmse:6.01847                                                   
[12]	validation-rmse:5.91803            

...

[0]	validation-rmse:5.44995                                                   
[1]	validation-rmse:5.25682                                                   
[2]	validation-rmse:5.24228                                                   
[3]	validation-rmse:5.23263                                                   
[4]	validation-rmse:5.22506                                                   
[5]	validation-rmse:5.21758                                                   
[6]	validation-rmse:5.20916                                                   
[7]	validation-rmse:5.20318                                                   
[8]	validation-rmse:5.20246                                                   
[9]	validation-rmse:5.20151                                                   
[10]	validation-rmse:5.20326                                                  
[11]	validation-rmse:5.20204                                                  
[12]	validation-rmse:5.19517                        

/Users/josesmac/MLOps_Projects/mlops_zoomcamp_24/zmcp/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:23:09] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.04621                                                   
[1]	validation-rmse:7.14456                                                   
[2]	validation-rmse:6.53462                                                   
[3]	validation-rmse:6.09961                                                   
[4]	validation-rmse:5.82510                                                   
[5]	validation-rmse:5.65235                                                   
[6]	validation-rmse:5.52242                                                   
[7]	validation-rmse:5.44645                                                   
[8]	validation-rmse:5.39664                                                   
[9]	validation-rmse:5.36181                                                   
[10]	validation-rmse:5.33960                                                  
[11]	validation-rmse:5.32296                                                  
[12]	validation-rmse:5.30336                        

/Users/josesmac/MLOps_Projects/mlops_zoomcamp_24/zmcp/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:23:32] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.00191                                                   
[1]	validation-rmse:8.70423                                                   
[2]	validation-rmse:8.42800                                                   
[3]	validation-rmse:8.17242                                                   
[4]	validation-rmse:7.93595                                                   
[5]	validation-rmse:7.71875                                                   
[6]	validation-rmse:7.51691                                                   
[7]	validation-rmse:7.33254                                                   
[8]	validation-rmse:7.16106                                                   
[9]	validation-rmse:7.00532                                                   
[10]	validation-rmse:6.86013                                                  
[11]	validation-rmse:6.72599                                                  
[12]	validation-rmse:6.60604                        

In [50]:
best_result

{'learning_rate': 0.3931914926020957,
 'max_depth': 5.0,
 'min_child_weight': 4.061140110039656,
 'reg_alpha': 0.0812130837749679,
 'reg_lambda': 0.010221480465528184}

In [49]:
mlflow.xgboost.autolog(disable=True)

In [53]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        "learning_rate": 0.39319149260209574,
        "max_depth": 5,
        "min_child_weight": 4.061140110039656,
        "objective": "reg:linear",
        "reg_alpha": 0.0812130837749679,
        "reg_lambda": 0.010221480465528184,
        "seed": 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("../models/hw2/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("../models/hw2/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:7.25975
[1]	validation-rmse:6.30296
[2]	validation-rmse:5.88128
[3]	validation-rmse:5.69972
[4]	validation-rmse:5.61293
[5]	validation-rmse:5.57003
[6]	validation-rmse:5.55271
[7]	validation-rmse:5.53824
[8]	validation-rmse:5.52051
[9]	validation-rmse:5.51369
[10]	validation-rmse:5.50921
[11]	validation-rmse:5.50488
[12]	validation-rmse:5.50072
[13]	validation-rmse:5.49646


/Users/josesmac/MLOps_Projects/mlops_zoomcamp_24/zmcp/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:30:17] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[14]	validation-rmse:5.49269
[15]	validation-rmse:5.48622
[16]	validation-rmse:5.48216
[17]	validation-rmse:5.48065
[18]	validation-rmse:5.47605
[19]	validation-rmse:5.47092
[20]	validation-rmse:5.46623
[21]	validation-rmse:5.46316
[22]	validation-rmse:5.46070
[23]	validation-rmse:5.45559
[24]	validation-rmse:5.45100
[25]	validation-rmse:5.44850
[26]	validation-rmse:5.44589
[27]	validation-rmse:5.44262
[28]	validation-rmse:5.43623
[29]	validation-rmse:5.43418
[30]	validation-rmse:5.43235
[31]	validation-rmse:5.42945
[32]	validation-rmse:5.42746
[33]	validation-rmse:5.42397
[34]	validation-rmse:5.42134
[35]	validation-rmse:5.41909
[36]	validation-rmse:5.41600
[37]	validation-rmse:5.41364
[38]	validation-rmse:5.41261
[39]	validation-rmse:5.41059
[40]	validation-rmse:5.40892
[41]	validation-rmse:5.40653
[42]	validation-rmse:5.40553
[43]	validation-rmse:5.40336
[44]	validation-rmse:5.40022
[45]	validation-rmse:5.39516
[46]	validation-rmse:5.39284
[47]	validation-rmse:5.39178
[48]	validatio

/Users/josesmac/MLOps_Projects/mlops_zoomcamp_24/zmcp/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:30:32] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [56]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "../data/hw2/train/green_tripdata_2023-01.parquet")
        mlflow.log_param("valid-data-path", "../data/hw2/train/green_tripdata_2023-02.parquet")
        mlflow.log_artifact("../models/hw2/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        

2024/05/27 17:41:42 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/27 17:47:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/27 17:47:24 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/27 17:56:40 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/Users/josesmac/MLOps_Projects/mlops_zoomcamp_24/zmcp/lib/python3.9/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/josesmac/MLOps_Projects/mlops_zoomcamp_24/zmcp/lib/python3.